In [1]:
#coding=utf-8
import tushare as ts
import pandas as pd
import talib
import time
import datetime
from opendatatools import swindex
import timestamp
from os import walk
ts.set_token('bc406c9fab768c6c699ef450685daed298c5abf2dbdba8da4e810cab')
pro = ts.pro_api()

today=20201031

In [2]:
import baostock as bs
import pandas as pd

# 登陆系统
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

# 获取指数(综合指数、规模指数、一级行业指数、二级行业指数、策略指数、成长指数、价值指数、主题指数)K线数据
# 综合指数，例如：sh.000001 上证指数，sz.399106 深证综指 等；
# 规模指数，例如：sh.000016 上证50，sh.000300 沪深300，sh.000905 中证500，sz.399001 深证成指等；
# 一级行业指数，例如：sh.000037 上证医药，sz.399433 国证交运 等；
# 二级行业指数，例如：sh.000952 300地产，sz.399951 300银行 等；
# 策略指数，例如：sh.000050 50等权，sh.000982 500等权 等；
# 成长指数，例如：sz.399376 小盘成长 等；
# 价值指数，例如：sh.000029 180价值 等；
# 主题指数，例如：sh.000015 红利指数，sh.000063 上证周期 等；

# 详细指标参数，参见“历史行情指标参数”章节
rs = bs.query_history_k_data_plus("000906.sh",
    "date,code,open,high,low,close,preclose,volume,amount,pctChg",
    start_date='2014-01-01', end_date='2020-10-31', frequency="d")
print('query_history_k_data_plus respond error_code:'+rs.error_code)
print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

# 打印结果集
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)
result.to_csv("C:/quant/index/history_Index_k_data.csv", index=False)# 登出系统
bs.logout()

login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!


In [3]:
#因为baostock出数据慢，手动输入当日数据后也能算MA

result=pd.read_csv("C:/quant/index/history_Index_k_data.csv")# 登出系统

ma5=talib.SMA(result.close,timeperiod=5)
ma10=talib.SMA(result.close,timeperiod=10)
ma20=talib.SMA(result.close,timeperiod=20)
ma30=talib.SMA(result.close,timeperiod=30)
ma60=talib.SMA(result.close,timeperiod=60)
ma120=talib.SMA(result.close,timeperiod=120)

ema5=talib.SMA(result.close,timeperiod=5)
ema10=talib.SMA(result.close,timeperiod=10)
ema20=talib.SMA(result.close,timeperiod=20)
ema30=talib.SMA(result.close,timeperiod=30)
ema60=talib.SMA(result.close,timeperiod=60)
ema120=talib.SMA(result.close,timeperiod=120)


result['MA5']=ma5
result['MA10']=ma10
result['MA20']=ma20
result['MA30']=ma30
result['MA60']=ma60
result['MA120']=ma120
result['EMA5']=ema5
result['EMA10']=ema10
result['EMA20']=ema20
result['EMA30']=ema30
result['EMA60']=ema60
result['EMA120']=ema120


# 结果集输出到csv文件
result.to_csv("C:/quant/index/history_Index_k_data.csv", index=False)
print(result)

# 登出系统
bs.logout()

            date       code       open       high        low      close  \
0     2014-01-02  sh.000906  2582.9100  2586.9400  2570.9500  2586.0200   
1     2014-01-03  sh.000906  2576.6100  2579.2900  2544.5900  2554.8000   
2     2014-01-06  sh.000906  2549.4700  2549.4700  2487.0700  2493.4000   
3     2014-01-07  sh.000906  2478.0200  2504.6600  2473.2700  2496.8600   
4     2014-01-08  sh.000906  2499.3500  2523.1300  2488.0000  2500.8800   
...          ...        ...        ...        ...        ...        ...   
1658  2020-10-26  sh.000906  4926.6937  4958.5038  4878.0131  4936.0554   
1659  2020-10-27  sh.000906  4922.0691  4952.3297  4912.0689  4943.8202   
1660  2020-10-28  sh.000906  4946.8813  4993.1433  4922.5219  4978.9404   
1661  2020-10-29  sh.000906  4921.3398  5037.3505  4916.0026  5009.2565   
1662  2020-10-30  sh.000906  5015.4587  5017.0833  4909.0489  4919.8791   

       preclose       volume        amount    pctChg  ...         MA20  \
0     2589.3500   7599455

In [4]:

pool = pro.stock_basic()
pool.head(5)
writer = pd.ExcelWriter('C:/quant/list/list.xlsx') 
pool.to_excel(writer,'Sheet1')
writer.save()
writer.close()


In [5]:

#从TUSHARE获取申万一级行业列表及其对应股票
Sdf1 = pro.index_classify(level='L1', src='SW')
Sdf1.to_csv('C:/quant/swsindex/L1.csv',index=False,encoding='utf-8-sig') 
#获取申万二级行业列表
Sdf2 = pro.index_classify(level='L2', src='SW')
Sdf2.to_csv('C:/quant/swsindex/L2.csv',index=False,encoding='utf-8-sig')

#获取申万三级级行业列表
Sdf3 = pro.index_classify(level='L3', src='SW')
Sdf3.dropna()
Sdf3['index_code']=Sdf3['index_code'].str.split('.', 2).str[0]

Sdf3.to_csv('C:/quant/swsindex/L3.csv',index=False,encoding='utf-8-sig') 
#Sdf3.head(10)
#SW1=pd.merge(Sdf1,Sdf2,how='right',on=['index_code','index_code'])
#SW2=pd.merge(SW1,Sdf2,how='right',on=['index_code','index_code'])


In [6]:
%%time
#3级行业对应个股
L3stocklist=pd.DataFrame()
for i in Sdf3.index_code:
    time.sleep(0.3)
    X=str(i)
    df23, msg = swindex.get_index_cons(X)
    df23['index_code']=X
    df23.rename(columns={'stock_code':'ts_code'}, inplace = True)
    df23=df23.drop(['start_date','weight'],axis=1)
    L3stocklist=L3stocklist.append(df23)
L3stocklist=pd.merge(L3stocklist,Sdf3,how='left',on='index_code')
L3stocklist

Wall time: 1min 22s


,ts_code,stock_name,index_code,industry_name,level
0,600759,洲际油气,850211,石油开采,L3
1,600777,新潮能源,850211,石油开采,L3
2,601857,中国石油,850211,石油开采,L3
3,000552,靖远煤电,850221,煤炭开采,L3
4,000571,*ST大洲,850221,煤炭开采,L3
...,...,...,...,...,...
3848,600811,东方集团,852311,综合,L3
3849,600817,ST宏盛,852311,综合,L3
3850,600818,中路股份,852311,综合,L3
3851,603060,国检集团,852311,综合,L3


In [7]:
%%time
#从申银端口通过刚才过滤的3级行业列表获取数据
allindex = pd.DataFrame() 
L3stocklist=pd.DataFrame()
for i in Sdf3.index_code:
    time.sleep(0.3)
    X=str(i)
    df2, msg = swindex.get_index_daily(X,start_date=20100101,end_date=today)
    df2.to_csv(r'C:/quant/indexdaydate/2020/'+X+'.csv',index=False,encoding='utf-8-sig') 
    df23, msg = swindex.get_index_cons(X)
    df23.rename(columns={'stock_code':'ts_code'}, inplace = True)
    df23=df23.drop(['start_date','weight'],axis=1)
    allindex = allindex.append(df2)
    L3stocklist=L3stocklist.append(df23)
allindex = allindex.sort_values(by=['index_code','date'],axis=0,ascending=[True,True])

   


http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('850211') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('850221') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('850222') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('850231') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('850241') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('850242') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('850311

http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('850614') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('850623') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('857221') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('857251') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('857231') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('857232') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('857233

http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('858811') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('851111') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('851112') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('851113') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('851114') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('851115') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('851121

http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('851242') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('851243') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('851244') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('852111') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('852112') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('852121') and BargainDate >= '20100101' and BargainDate <= '20201031' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('852131

KeyboardInterrupt: 

In [8]:
L3stocklist

,ts_code,stock_name
0,600759,洲际油气
1,600777,新潮能源
2,601857,中国石油
0,000552,靖远煤电
1,000571,*ST大洲
...,...,...
14,601018,宁波港
15,601228,广州港
16,601298,青岛港
17,601326,秦港股份


In [9]:
#申银源数据格式不对，转换格式
allindex['open']=allindex['open'].astype("float")
allindex['high']=allindex['high'].astype("float")
allindex['low']=allindex['low'].astype("float")
allindex['close']=allindex['close'].astype("float")
allindex['vol']=allindex['vol'].astype("float")
allindex['amount']=allindex['amount'].astype("float")
allindex['change_pct']=allindex['change_pct'].astype("float")
allindex.isnull().sum()


index_code    0
index_name    0
date          0
open          0
high          0
low           0
close         0
vol           0
amount        0
change_pct    0
dtype: int64

In [10]:
#计算申银行业指数MA，MACD
from scipy.stats import rankdata
swsf=allindex[allindex['index_code'].notna()]

close=swsf.close.values
swsf.date=swsf.date.values

diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
SMA5=talib.SMA(close,timeperiod=5)
SMA10=talib.SMA(close,timeperiod=10)
SMA20=talib.SMA(close,timeperiod=20)
SMA60=talib.SMA(close,timeperiod=60)
SMA120=talib.SMA(close,timeperiod=120)
SMA350=talib.SMA(close,timeperiod=350)

EMA5=talib.EMA(close,timeperiod=5)
EMA10=talib.EMA(close,timeperiod=10)
EMA20=talib.EMA(close,timeperiod=20)
EMA60=talib.EMA(close,timeperiod=60)
EMA120=talib.EMA(close,timeperiod=120)

swsf['diff']=diff
swsf['dea']=dea
swsf['MACD']=my_macd
swsf['SMA5']=SMA5
swsf['SMA10']=SMA10
swsf['SMA20']=SMA20
swsf['SMA60']=SMA60
swsf['SMA120']=SMA120
swsf['SMA350']=SMA350

swsf['EMA5']=EMA5
swsf['EMA10']=EMA10
swsf['EMA60']=EMA60
swsf['EMA120']=EMA120

swsf['EMA20']=EMA20
swsf['SMACD-K']=(diff-dea)/dea
swsf['C/S']=(close/SMA20-1)*100
swsf['M/L']=(SMA60/SMA120-1)*100
swsf['S/M']=(SMA20/SMA60-1)*100
swsf['BreakSignal-20D']=close/EMA20-1
swsf.reset_index(drop=True, inplace=True)

swsf['dailyamount']=swsf.groupby('date')['amount'].sum()
swsf['indexamount']=swsf.amount/swsf.dailyamount


swsf.loc[swsf['index_code'] == swsf['index_code'].shift(1),'Chg1D']= (swsf.close/swsf.close.shift(1)-1)*100
swsf.loc[swsf['index_code'] == swsf['index_code'].shift(4),'Chg5D']= (swsf.close/swsf.close.shift(4)-1)*100
swsf.loc[swsf['index_code'] == swsf['index_code'].shift(19),'Chg20D']= (swsf.close/swsf.close.shift(19)-1)*100
swsf.loc[swsf['index_code'] == swsf['index_code'].shift(59),'Chg60D']= (swsf.close/swsf.close.shift(59)-1)*100
swsf.loc[swsf['index_code'] == swsf['index_code'].shift(119),'Chg120D']= (swsf.close/swsf.close.shift(119)-1)*100

swsf.fillna(0,inplace=True) #去除null
swsf.loc[(swsf['close']>swsf['EMA20']) & (swsf['close']>swsf['SMA20']) & (swsf['SMA20']>swsf['SMA20'].shift(19)),'K1']=1 #破线拐头
swsf.loc[(swsf['EMA20']>swsf['EMA60']) & (swsf['close']>swsf['EMA20'].shift(19)),'K2']=1 #双均线交叉
swsf.loc[(swsf['EMA20']>swsf['SMA60']) & (swsf['close']>swsf['EMA20'].shift(19)),'K3']=1 
swsf.loc[(swsf['SMA20']>swsf['EMA60']) & (swsf['close']>swsf['SMA20'].shift(19)),'K4']=1 
swsf.loc[(swsf['SMA20']>swsf['SMA60']) & (swsf['close']>swsf['SMA20'].shift(19)),'K5']=1 

swsf.loc[(swsf['close']<swsf['EMA20']) & (swsf['close']<swsf['SMA20']) & (swsf['SMA20']<swsf['SMA20'].shift(19)),'K1']=-1 #空头方向破线拐头
swsf.loc[(swsf['EMA20']<swsf['EMA60']) & (swsf['close']<swsf['EMA20'].shift(19)),'K2']=-1 #双均线交叉
swsf.loc[(swsf['EMA20']<swsf['SMA60']) & (swsf['close']<swsf['EMA20'].shift(19)),'K3']=-1 
swsf.loc[(swsf['SMA20']<swsf['EMA60']) & (swsf['close']<swsf['SMA20'].shift(19)),'K4']=-1 
swsf.loc[(swsf['SMA20']<swsf['SMA60']) & (swsf['close']<swsf['SMA20'].shift(19)),'K5']=-1 
swsf.fillna(0,inplace=True) #去除null
swsf['Indextrendscore-L']=(swsf.K1+swsf.K2+swsf.K3+swsf.K4+swsf.K5)

swsf.head(10)
#swsf.to_csv('C:/quant/index/SWoutput.csv',index=False,encoding='utf-8-sig') 

,index_code,index_name,date,open,high,low,close,vol,amount,change_pct,...,Chg5D,Chg20D,Chg60D,Chg120D,K1,K2,K3,K4,K5,Indextrendscore-L
0,850211,石油开采,2020-10-30,200.51,201.87,196.32,197.33,2.12,5.81,-1.36,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,850211,石油开采,2020-10-29,200.77,201.12,199.29,200.05,1.13,3.43,-0.99,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,850211,石油开采,2020-10-28,202.75,203.05,200.31,202.05,0.98,2.91,-0.55,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,850211,石油开采,2020-10-27,202.96,203.72,201.88,203.16,0.83,2.57,-0.20,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,850211,石油开采,2020-10-26,204.24,204.55,202.61,203.57,0.97,3.05,-0.48,...,3.162216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,850211,石油开采,2020-10-23,204.40,206.72,203.94,204.55,1.19,4.01,0.13,...,2.249438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,850211,石油开采,2020-10-22,204.40,204.81,202.88,204.29,0.72,2.09,-0.25,...,1.108636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,850211,石油开采,2020-10-21,205.53,206.18,203.44,204.81,0.96,2.87,-0.35,...,0.812168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,850211,石油开采,2020-10-20,205.42,205.88,203.79,205.53,0.77,2.24,-0.09,...,0.962814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,850211,石油开采,2020-10-19,206.53,208.66,204.96,205.72,1.34,4.12,-0.22,...,0.571987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
BBIndex=allindex[allindex['index_code'].notna()]
BBIndex.loc[BBIndex.close>=BBIndex.open,'Signal']=0
BBIndex.loc[BBIndex.close<BBIndex.open,'Signal']=1
BBIndex['ChangeSignal']=BBIndex.Signal.diff().abs()
BBIndex.ChangeSignal.fillna(0,inplace=True) #解决首行0
BBIndex['BBperiod']=BBIndex.groupby('index_code').ChangeSignal.cumsum()

#板块级别日线涨跌周期

high = BBIndex.groupby(['index_name','BBperiod'])['high'].max().rename('high').reset_index()
low = BBIndex.groupby(['index_name','BBperiod'])['low'].min().rename('low').reset_index()
amount = BBIndex.groupby(['index_name','BBperiod'])['amount'].sum().rename('amount').reset_index()
open1 = BBIndex.groupby(['index_name','BBperiod'])['open'].first().rename('open').reset_index()
close1 = BBIndex.groupby(['index_name','BBperiod'])['close'].last().rename('close').reset_index()
lastperiod=BBIndex.groupby('index_name').BBperiod.max().rename('lastbar')

BBI2 = pd.merge(open1,close1,how='left',on=('index_name','BBperiod')) 
BBI2 = pd.merge(BBI2,high,how='left',on=('index_name','BBperiod')) 
BBI2 = pd.merge(BBI2,low,how='left',on=('index_name','BBperiod')) 
BBI2 = pd.merge(BBI2,amount,how='left',on=('index_name','BBperiod')) 
BBI2 = pd.merge(BBI2,lastperiod,how='left',on='index_name') 

close=BBI2.close.values
#swsf.date=swsf.date.values

diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
BBI2['D-diff']=diff
BBI2['D-dea']=dea
BBI2['D-MACD']=my_macd
BBI2=BBI2[BBI2['BBperiod']==BBI2['lastbar']]
BBI2 = BBI2.sort_values(by=['D-MACD'],axis=0,ascending=[False])

#BBI2.to_csv('C:/Users/atianth/Desktop/BBtest2.csv',index=False,encoding='utf-8-sig') 
BBI2

,index_name,BBperiod,open,close,high,low,amount,lastbar,D-diff,D-dea,D-MACD
87717,煤炭开采,1332.0,7101.66,6988.05,7139.59,6942.49,47.10,1332.0,328.925232,225.518144,103.407089
3871,乘用车,1313.0,6362.85,6409.92,6500.86,6300.62,23.77,1313.0,262.627500,161.056762,101.570738
48850,建筑机械,1338.0,9174.00,9163.25,9320.38,8994.16,36.32,1338.0,189.327682,105.308027,84.019656
90250,燃气,1287.0,5609.82,5539.29,5890.31,5409.98,34.62,1287.0,244.315439,175.313948,69.001491
61315,果蔬生产,1240.0,8807.81,8866.53,8976.63,8705.05,0.53,1240.0,33.940529,-29.802691,63.743220
...,...,...,...,...,...,...,...,...,...,...,...
67434,氮肥,1196.0,4550.14,4478.13,4638.06,4411.48,57.57,1196.0,144.679029,169.838815,-25.159786
9790,其他建材,1198.0,3530.74,3505.76,3586.02,3453.52,10.91,1198.0,-46.804308,-20.961294,-25.843014
82607,港口,1304.0,4851.13,4819.98,4884.15,4800.77,10.14,1304.0,59.997589,86.628052,-26.630463
125215,酒店,1230.0,2270.73,2240.79,2315.19,2231.73,4.69,1230.0,-50.717819,-18.875976,-31.841844


In [12]:
BBWIndex=allindex[allindex['index_code'].notna()]
BBWIndex['year']=BBWIndex.date.dt.isocalendar().year
BBWIndex['week']=BBWIndex.date.dt.isocalendar().week
high = BBWIndex.groupby(['index_name','year','week'])['high'].max().rename('high').reset_index()
low = BBWIndex.groupby(['index_name','year','week'])['low'].min().rename('low').reset_index()
amount = BBWIndex.groupby(['index_name','year','week'])['amount'].sum().rename('amount').reset_index()
open1 = BBWIndex.groupby(['index_name','year','week'])['open'].first().rename('open').reset_index()
close1 = BBWIndex.groupby(['index_name','year','week'])['close'].last().rename('close').reset_index()

BBWI2 = pd.merge(open1,close1,how='left',on=('index_name','year','week')) 
BBWI2 = pd.merge(BBWI2,high,how='left',on=('index_name','year','week')) 
BBWI2 = pd.merge(BBWI2,low,how='left',on=('index_name','year','week')) 
BBWI2 = pd.merge(BBWI2,amount,how='left',on=('index_name','year','week')) 

BBWI2.loc[BBWI2.close>=BBWI2.open,'Signal']=0
BBWI2.loc[BBWI2.close<BBWI2.open,'Signal']=1
BBWI2['ChangeSignal']=BBWI2.Signal.diff().abs()
BBWI2.ChangeSignal.fillna(0,inplace=True) #解决首行0
BBWI2['BBperiod']=BBWI2.groupby('index_name').ChangeSignal.cumsum()

high2 = BBWI2.groupby(['index_name','BBperiod'])['high'].max().rename('high').reset_index()
low2 = BBWI2.groupby(['index_name','BBperiod'])['low'].min().rename('low').reset_index()
amount2 = BBWI2.groupby(['index_name','BBperiod'])['amount'].sum().rename('amount').reset_index()
open2 = BBWI2.groupby(['index_name','BBperiod'])['open'].first().rename('open').reset_index()
close2 = BBWI2.groupby(['index_name','BBperiod'])['close'].last().rename('close').reset_index()
lastperiod=BBWI2.groupby('index_name').BBperiod.max().rename('lastbar')

BBWI3 = pd.merge(open2,close2,how='left',on=('index_name','BBperiod')) 
BBWI3 = pd.merge(BBWI3,high2,how='left',on=('index_name','BBperiod')) 
BBWI3 = pd.merge(BBWI3,low2,how='left',on=('index_name','BBperiod')) 
BBWI3 = pd.merge(BBWI3,amount2,how='left',on=('index_name','BBperiod')) 
BBWI3 = pd.merge(BBWI3,lastperiod,how='left',on='index_name') 

close=BBWI3.close.values
#swsf.date=swsf.date.values

diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
BBWI3['W-diff']=diff
BBWI3['W-dea']=dea
BBWI3['W-MACD']=my_macd
BBWI3=BBWI3[BBWI3['BBperiod']==BBWI3['lastbar']]
BBWI3 = BBWI3.sort_values(by=['W-MACD'],axis=0,ascending=[False])

BBWI3

,index_name,BBperiod,open,close,high,low,amount,lastbar,W-diff,W-dea,W-MACD
22689,白酒,290.0,67148.92,64158.34,68750.14,61879.77,2126.66,290.0,7210.355563,6591.398078,618.957485
11855,旅游综合,285.0,13108.96,12144.26,13261.81,11766.13,152.16,285.0,1889.131043,1471.848703,417.282340
17995,洗衣机,289.0,10837.92,11578.40,11830.84,10366.71,2.15,289.0,1006.167530,645.711413,360.456116
24948,空调,283.0,43842.15,44231.90,46624.68,41766.86,664.55,283.0,1999.496873,1694.198488,305.298385
891,乘用车,310.0,8202.70,8547.28,9822.96,7675.70,1645.31,310.0,582.033679,348.500437,233.533242
...,...,...,...,...,...,...,...,...,...,...,...
31117,饲料,275.0,7931.54,7559.48,8011.97,7287.98,185.76,275.0,631.194004,692.191951,-60.997947
2307,其他建材,296.0,9933.41,9515.25,10077.35,9183.12,203.79,296.0,964.278092,1039.432936,-75.154844
7795,医疗器械,310.0,11292.98,10771.84,11522.32,10435.34,1043.68,310.0,1144.692905,1225.599657,-80.906752
16838,水泥制造,291.0,8384.78,8361.37,8494.13,8139.78,173.88,291.0,397.485800,511.240545,-113.754745


In [13]:
BBMIndex=allindex[allindex['index_code'].notna()]
BBMIndex['year']=BBMIndex.date.dt.isocalendar().year
BBMIndex['month']=BBMIndex.date.dt.month
high = BBMIndex.groupby(['index_name','year','month'])['high'].max().rename('high').reset_index()
low = BBMIndex.groupby(['index_name','year','month'])['low'].min().rename('low').reset_index()
amount = BBMIndex.groupby(['index_name','year','month'])['amount'].sum().rename('amount').reset_index()
open1 = BBMIndex.groupby(['index_name','year','month'])['open'].first().rename('open').reset_index()
close1 = BBMIndex.groupby(['index_name','year','month'])['close'].last().rename('close').reset_index()

BBMI2 = pd.merge(open1,close1,how='left',on=('index_name','year','month')) 
BBMI2 = pd.merge(BBMI2,high,how='left',on=('index_name','year','month')) 
BBMI2 = pd.merge(BBMI2,low,how='left',on=('index_name','year','month')) 
BBMI2 = pd.merge(BBMI2,amount,how='left',on=('index_name','year','month')) 

BBMI2.loc[BBMI2.close>=BBMI2.open,'Signal']=0
BBMI2.loc[BBMI2.close<BBMI2.open,'Signal']=1
BBMI2['ChangeSignal']=BBMI2.Signal.diff().abs()
BBMI2.ChangeSignal.fillna(0,inplace=True) #解决首行0
BBMI2['BBperiod']=BBMI2.groupby('index_name').ChangeSignal.cumsum()

high2 = BBMI2.groupby(['index_name','BBperiod'])['high'].max().rename('high').reset_index()
low2 = BBMI2.groupby(['index_name','BBperiod'])['low'].min().rename('low').reset_index()
amount2 = BBMI2.groupby(['index_name','BBperiod'])['amount'].sum().rename('amount').reset_index()
open2 = BBMI2.groupby(['index_name','BBperiod'])['open'].first().rename('open').reset_index()
close2 = BBMI2.groupby(['index_name','BBperiod'])['close'].last().rename('close').reset_index()
lastperiod=BBMI2.groupby('index_name').BBperiod.max().rename('lastbar')

BBMI3 = pd.merge(open2,close2,how='left',on=('index_name','BBperiod')) 
BBMI3 = pd.merge(BBMI3,high2,how='left',on=('index_name','BBperiod')) 
BBMI3 = pd.merge(BBMI3,low2,how='left',on=('index_name','BBperiod')) 
BBMI3 = pd.merge(BBMI3,amount2,how='left',on=('index_name','BBperiod')) 
BBMI3 = pd.merge(BBMI3,lastperiod,how='left',on='index_name') 

close=BBMI3.close.values
#swsf.date=swsf.date.values

diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
BBMI3['M-diff']=diff
BBMI3['M-dea']=dea
BBMI3['M-MACD']=my_macd
BBMI3=BBMI3[BBMI3['BBperiod']==BBMI3['lastbar']]
BBMI3 = BBMI3.sort_values(by=['M-MACD'],axis=0,ascending=[False])

BBMI3

,index_name,BBperiod,open,close,high,low,amount,lastbar,M-diff,M-dea,M-MACD
4929,白酒,74.0,42377.91,42387.41,42713.96,41308.34,280.06,74.0,9386.485363,8299.766023,1086.719340
266,人工景点,65.0,57973.52,57788.27,58533.75,57379.95,3.44,65.0,6530.843139,6044.787257,486.055882
2572,旅游综合,65.0,13108.96,6643.46,14526.15,6524.96,596.87,65.0,1462.975804,1050.400665,412.575139
2447,建筑机械,67.0,10576.46,10560.25,10809.36,10329.71,56.98,67.0,1334.547235,972.566470,361.980765
4795,生物制品,64.0,8571.42,8555.81,8892.68,8389.26,134.52,64.0,1287.570411,945.910330,341.660082
...,...,...,...,...,...,...,...,...,...,...,...
4159,港口,65.0,3671.24,3822.42,3960.02,3384.42,726.23,65.0,-301.061380,-232.383212,-68.678167
6515,金属新材料,58.0,3349.26,2878.04,3614.60,2808.07,1116.27,58.0,-257.715244,-150.615309,-107.099936
6648,铁路设备,63.0,5955.85,7036.51,7124.64,5767.06,345.17,63.0,-830.000774,-676.889143,-153.111631
3892,洗衣机,61.0,12217.90,8246.24,20682.93,8010.64,63.04,61.0,640.720711,815.002301,-174.281591


In [14]:
IndexReport=swsf[['date','index_name','close','Chg1D','Chg5D','Chg20D','Chg60D','Chg120D','C/S','S/M','M/L','K1','Indextrendscore-L']]
lastday=IndexReport.date.max()
IndexReport=IndexReport[IndexReport.date==lastday]
IDReport=BBI2[['index_name','D-diff','D-dea','D-MACD']]
IWReport=BBWI3[['index_name','W-diff','W-dea','W-MACD']]
IMReport=BBMI3[['index_name','M-diff','M-dea','M-MACD']]
IndexReport=pd.merge(IndexReport,IDReport,how='left',on='index_name')
IndexReport=pd.merge(IndexReport,IWReport,how='left',on='index_name')
IndexReport=pd.merge(IndexReport,IMReport,how='left',on='index_name')


IndexReport
IndexReport.to_csv('C:/quant/output/Indexoutput.csv',index=False,encoding='utf-8-sig') 

In [15]:

#从TUSHARE端口获取成分股
ISindex = pd.DataFrame() 

for i in Sdf1.index_code:
    time.sleep(0.7)
    Tf=pro.index_member(index_code=i)    
    ISindex = ISindex.append(Tf)

ISindex.to_csv(r'C:/quant/swsindex/total.csv') 
ISindex.rename(columns={'con_code':'ts_code'}, inplace = True)
ISindex=pd.merge(ISindex,Sdf1,how='left',on='index_code')
ISindex=ISindex.drop(["in_date",'out_date','level'],axis=1)

ISindex
    


,index_code,ts_code,industry_name
0,801020.SI,600348.SH,采掘
1,801020.SI,600121.SH,采掘
2,801020.SI,600157.SH,采掘
3,801020.SI,600792.SH,采掘
4,801020.SI,600123.SH,采掘
...,...,...,...
3730,801230.SI,600175.SH,综合
3731,801230.SI,600149.SH,综合
3732,801230.SI,600234.SH,综合
3733,801230.SI,300688.SZ,综合


In [16]:
latestday=max(swsf.date)
ETF=swsf[swsf.date==latestday]
ETF=ETF[['index_code','K1','Indextrendscore-L']]
ISindex['index_code']=ISindex['index_code'].str.split('.', 2).str[0]

ISindex2 = pd.merge(ISindex, ETF, how='left',on='index_code') #分段信息
ISindex2

,index_code,ts_code,industry_name,K1,Indextrendscore-L
0,801020,600348.SH,采掘,NaN,NaN
1,801020,600121.SH,采掘,NaN,NaN
2,801020,600157.SH,采掘,NaN,NaN
3,801020,600792.SH,采掘,NaN,NaN
4,801020,600123.SH,采掘,NaN,NaN
...,...,...,...,...,...
3730,801230,600175.SH,综合,NaN,NaN
3731,801230,600149.SH,综合,NaN,NaN
3732,801230,600234.SH,综合,NaN,NaN
3733,801230,300688.SZ,综合,NaN,NaN


In [17]:
#tushare交易日历
calendar = pro.query('trade_cal', start_date='20201028', end_date=today)
calendar = calendar.drop(calendar[calendar['is_open']==0].index)
calendar

,exchange,cal_date,is_open
0,SSE,20201028,1
1,SSE,20201029,1
2,SSE,20201030,1


In [18]:

#从tushare下载日数据

for i in calendar.cal_date:  #range(20201021,today+1):
    time.sleep(0.3)
    print (i) 
    x=str(i)
    df = pro.daily(trade_date=x) #个股数据
    Adf = pro.query('adj_factor', trade_date=x) #复权因子
    Adf.rename(columns={'trade_date':'trade_date_x'}, inplace = True)
    Adf.adj_factor.fillna(0,inplace=True)


    Mdf = pro.moneyflow(trade_date=x) #个股资金流向，出韭菜指数
    X1=pd.merge(df,Mdf,on='ts_code')
    X1=pd.merge(X1,Adf,on=['ts_code','trade_date_x'])

    mf = pro.margin(trade_date=x)#融资融券数据
    GCdf = pro.repo_daily(ts_code='204001.sh',trade_date=x) # 逆回购数据
    #HSdf = pro.repo_daily(ts_code='000906.sh',trade_date=x) # 逆回购数据

    #X2=pd.merge(mf,GCdf,on='trade_date')

    #df2 = pro.index_dailybasic(start_date=x,fields='ts_code,trade_date,turnover_rate,pe,pe_ttm')
    #pe = df2['pe_ttm']


    writer1 = X1.to_csv('C:/quant/daydate/temp/'+x + '.csv') 
    writer2 = GCdf.to_csv('C:/quant/GCdata/2020/'+x + '.csv') 
    writer3 = mf.to_csv('C:/quant/financing/2020/'+x + '.csv') #要晚一天出，所以不能合并
    #writer4 = HSdf.to_csv('C:/quant/index/2020/'+x + '.csv') #要晚一天出，所以不能合并


   # df.to_csv(writer)
  #  writer.save()
   # writer.close()
    


20201028
20201029
20201030


In [19]:
'''#生成15-20年单年数据
import pandas as pd
import datetime
import numpy as np
from os import walk
for root,dirs,files in walk(r'C:/quant/daydate/2020/',topdown=False):
    num = len(files)
    alldata = pd.DataFrame() 
for i in range(num):
    newdata = pd.read_csv(r'C:/quant/daydate/2020/%s'%files[i])
    
    alldata = alldata.append(newdata)
alldata = alldata.sort_values(by=['ts_code','trade_date_x'],axis=0,ascending=[True,True])
alldata.reset_index(drop=True, inplace=True)
alldata.to_csv('C:/quant/daydate/summary/2020summary.csv') 
'''

"\nimport pandas as pd\nimport datetime\nimport numpy as np\nfrom os import walk\nfor root,dirs,files in walk(r'C:/quant/daydate/2020/',topdown=False):\n    num = len(files)\n    alldata = pd.DataFrame() \nfor i in range(num):\n    newdata = pd.read_csv(r'C:/quant/daydate/2020/%s'%files[i])\n    \n    alldata = alldata.append(newdata)\nalldata = alldata.sort_values(by=['ts_code','trade_date_x'],axis=0,ascending=[True,True])\nalldata.reset_index(drop=True, inplace=True)\nalldata.to_csv('C:/quant/daydate/summary/2020summary.csv') \n"

In [38]:
'''
%%time
#生成totalsummary原始版

for root,dirs,files in walk(r'C:/quant/daydate/summary',topdown=False):
    num = len(files)
    Salldata = pd.DataFrame()

for i in range(num):
    newdata = pd.read_csv(r'C:/quant/daydate/summary/%s'%files[i])
    newdata = newdata.drop(newdata.columns[0],axis=1) #去除tushare源生数据首列index

    Salldata = Salldata.append(newdata)
Salldata = Salldata.sort_values(by=['ts_code','trade_date_x'],axis=0,ascending=[True,True])
Salldata.reset_index(drop=True, inplace=True)
Salldata.drop_duplicates()
Salldata.to_csv('C:/quant/daydate/summary/totalsummary.csv') 
Salldata
'''

Wall time: 4min 13s


,ts_code,trade_date_x,open,high,low,close,pre_close,change,pct_chg,vol,...,sell_lg_vol,sell_lg_amount,buy_elg_vol,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount,adj_factor,Unnamed: 0.1
0,000001.SZ,20170103.0,9.11,9.18,9.09,9.16,9.10,0.06,0.6600,459840.49,...,142781.0,13065.97,113061.0,10343.41,130706.0,11939.93,71350.0,6531.67,104.758,NaN
1,000001.SZ,20170104.0,9.15,9.18,9.14,9.16,9.16,0.00,0.0000,449329.53,...,128951.0,11810.56,104137.0,9540.33,176725.0,16185.19,-6884.0,-608.63,104.758,NaN
2,000001.SZ,20170105.0,9.17,9.18,9.15,9.17,9.16,0.01,0.1100,344372.91,...,111251.0,10200.48,59032.0,5413.23,66840.0,6128.81,17325.0,1604.98,104.758,NaN
3,000001.SZ,20170106.0,9.17,9.17,9.11,9.13,9.17,-0.04,-0.4400,358154.20,...,96851.0,8851.30,94131.0,8599.49,75682.0,6910.19,-137025.0,-12502.51,104.758,NaN
4,000001.SZ,20170109.0,9.13,9.17,9.11,9.15,9.13,0.02,0.2200,361081.57,...,102405.0,9358.58,116900.0,10686.08,96489.0,8817.27,-38202.0,-3471.15,104.758,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213693,688981.SH,20201026.0,56.89,58.80,56.26,58.45,57.22,1.23,2.1496,322355.87,...,113528.0,65882.41,59181.0,34338.13,50715.0,29431.82,64816.0,37819.53,1.000,1348.0
3213694,688981.SH,20201027.0,58.00,59.02,57.50,58.70,58.45,0.25,0.4277,290649.23,...,95529.0,55806.10,46093.0,26955.05,65695.0,38389.14,15890.0,9525.46,1.000,2462.0
3213695,688981.SH,20201028.0,58.55,59.59,57.81,59.33,58.70,0.63,1.0733,330595.53,...,98312.0,57803.20,52588.0,31065.54,61195.0,36028.92,32511.0,19398.42,1.000,2002.0
3213696,688981.SH,20201029.0,58.55,61.32,58.40,60.88,59.33,1.55,2.6125,481297.87,...,151557.0,90516.65,150974.0,90569.41,92901.0,55291.97,89283.0,54031.13,1.000,2827.0


In [39]:
%%time
for root,dirs,files in walk(r'C:/quant/daydate/temp',topdown=False):
    num = len(files)
    alldata = pd.read_csv(r'C:/quant/daydate/summary/totalsummary.csv')

for i in range(num):
    newdata = pd.read_csv(r'C:/quant/daydate/temp/%s'%files[i])
    newdata = newdata.drop(newdata.columns[0],axis=1) #去除tushare源生数据首列index
    newdata=newdata.dropna(subset=["trade_date_x"])
    alldata = alldata.append(newdata)
alldata = alldata.dropna(subset=['trade_date_x'])
alldata = alldata.sort_values(by=['ts_code','trade_date_x'],axis=0,ascending=[True,True])
alldata.reset_index(drop=True, inplace=True)
alldata.to_csv('C:/quant/daydate/summary/totalsummary.csv') 
alldata

Wall time: 3min 40s


,Unnamed: 0,ts_code,trade_date_x,open,high,low,close,pre_close,change,pct_chg,...,sell_lg_vol,sell_lg_amount,buy_elg_vol,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount,adj_factor,Unnamed: 0.1
0,0,000001.SZ,20170103.0,9.11,9.18,9.09,9.16,9.10,0.06,0.6600,...,142781.0,13065.97,113061.0,10343.41,130706.0,11939.93,71350.0,6531.67,104.758,NaN
1,1,000001.SZ,20170104.0,9.15,9.18,9.14,9.16,9.16,0.00,0.0000,...,128951.0,11810.56,104137.0,9540.33,176725.0,16185.19,-6884.0,-608.63,104.758,NaN
2,2,000001.SZ,20170105.0,9.17,9.18,9.15,9.17,9.16,0.01,0.1100,...,111251.0,10200.48,59032.0,5413.23,66840.0,6128.81,17325.0,1604.98,104.758,NaN
3,3,000001.SZ,20170106.0,9.17,9.17,9.11,9.13,9.17,-0.04,-0.4400,...,96851.0,8851.30,94131.0,8599.49,75682.0,6910.19,-137025.0,-12502.51,104.758,NaN
4,4,000001.SZ,20170109.0,9.13,9.17,9.11,9.15,9.13,0.02,0.2200,...,102405.0,9358.58,116900.0,10686.08,96489.0,8817.27,-38202.0,-3471.15,104.758,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3209837,3213693,688981.SH,20201026.0,56.89,58.80,56.26,58.45,57.22,1.23,2.1496,...,113528.0,65882.41,59181.0,34338.13,50715.0,29431.82,64816.0,37819.53,1.000,1348.0
3209838,3213694,688981.SH,20201027.0,58.00,59.02,57.50,58.70,58.45,0.25,0.4277,...,95529.0,55806.10,46093.0,26955.05,65695.0,38389.14,15890.0,9525.46,1.000,2462.0
3209839,3213695,688981.SH,20201028.0,58.55,59.59,57.81,59.33,58.70,0.63,1.0733,...,98312.0,57803.20,52588.0,31065.54,61195.0,36028.92,32511.0,19398.42,1.000,2002.0
3209840,3213696,688981.SH,20201029.0,58.55,61.32,58.40,60.88,59.33,1.55,2.6125,...,151557.0,90516.65,150974.0,90569.41,92901.0,55291.97,89283.0,54031.13,1.000,2827.0


In [40]:
%%time
S=alldata
pd.isnull(S).sum()
S['date']=S.trade_date_x.astype(str)
S.date=S.date.str.split('.', 2).str[0]
S.date=pd.to_datetime(S.date)
S.open=S.open.fillna(method='ffill')
S.close=S.close.fillna(method='ffill') #防停牌无数据
S.high=S.high.fillna(method='ffill')
S.low=S.low.fillna(method='ffill')
#S.amount=S.amount.fillna(0,inplace=True)


latestday=S.date.max()
S.adj_factor.fillna(1,inplace=True)
S.loc[S['date']==latestday,'now_adj']=S['adj_factor'] # 最新复权因子
S.now_adj=S.now_adj.fillna(method='bfill')

#S.loc[S['ts_code']==S['ts_code'].shift(1),'now_adj']=S.now_adj.shift(1) # 最新复权因子

S['close_qfq'] = S['close'] * S['adj_factor'] / S.now_adj  # 前复权 当日收盘价 × 当日复权因子 / 最新复权因子
#alldata.to_csv('C:/quant/index/2020/test.csv') 

S

Wall time: 15.4 s


,Unnamed: 0,ts_code,trade_date_x,open,high,low,close,pre_close,change,pct_chg,...,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount,adj_factor,Unnamed: 0.1,date,now_adj,close_qfq
0,0,000001.SZ,20170103.0,9.11,9.18,9.09,9.16,9.10,0.06,0.6600,...,10343.41,130706.0,11939.93,71350.0,6531.67,104.758,NaN,2017-01-03,111.048,8.641158
1,1,000001.SZ,20170104.0,9.15,9.18,9.14,9.16,9.16,0.00,0.0000,...,9540.33,176725.0,16185.19,-6884.0,-608.63,104.758,NaN,2017-01-04,111.048,8.641158
2,2,000001.SZ,20170105.0,9.17,9.18,9.15,9.17,9.16,0.01,0.1100,...,5413.23,66840.0,6128.81,17325.0,1604.98,104.758,NaN,2017-01-05,111.048,8.650591
3,3,000001.SZ,20170106.0,9.17,9.17,9.11,9.13,9.17,-0.04,-0.4400,...,8599.49,75682.0,6910.19,-137025.0,-12502.51,104.758,NaN,2017-01-06,111.048,8.612857
4,4,000001.SZ,20170109.0,9.13,9.17,9.11,9.15,9.13,0.02,0.2200,...,10686.08,96489.0,8817.27,-38202.0,-3471.15,104.758,NaN,2017-01-09,111.048,8.631724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3209837,3213693,688981.SH,20201026.0,56.89,58.80,56.26,58.45,57.22,1.23,2.1496,...,34338.13,50715.0,29431.82,64816.0,37819.53,1.000,1348.0,2020-10-26,1.000,58.450000
3209838,3213694,688981.SH,20201027.0,58.00,59.02,57.50,58.70,58.45,0.25,0.4277,...,26955.05,65695.0,38389.14,15890.0,9525.46,1.000,2462.0,2020-10-27,1.000,58.700000
3209839,3213695,688981.SH,20201028.0,58.55,59.59,57.81,59.33,58.70,0.63,1.0733,...,31065.54,61195.0,36028.92,32511.0,19398.42,1.000,2002.0,2020-10-28,1.000,59.330000
3209840,3213696,688981.SH,20201029.0,58.55,61.32,58.40,60.88,59.33,1.55,2.6125,...,90569.41,92901.0,55291.97,89283.0,54031.13,1.000,2827.0,2020-10-29,1.000,60.880000


In [41]:
%%time
from scipy.stats import rankdata
df=S[S['ts_code'].notna()]

close=df.close_qfq
diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
SMA5=talib.SMA(close,timeperiod=5)
SMA10=talib.SMA(close,timeperiod=10)
SMA20=talib.SMA(close,timeperiod=20)
SMA60=talib.SMA(close,timeperiod=60)
SMA120=talib.SMA(close,timeperiod=120)

EMA5=talib.EMA(close,timeperiod=5)
EMA10=talib.EMA(close,timeperiod=10)
EMA20=talib.EMA(close,timeperiod=20)
EMA60=talib.EMA(close,timeperiod=60)
EMA120=talib.EMA(close,timeperiod=120)

df['diff']=diff
df['dea']=dea
df['MACD']=my_macd
df['SMA5']=SMA5
df['SMA10']=SMA10
df['SMA20']=SMA20
df['SMA60']=SMA60
df['SMA120']=SMA120

df['EMA5']=EMA5
df['EMA10']=EMA10
df['EMA60']=EMA60
df['EMA120']=EMA120

df['EMA20']=EMA20
df['SMACD-K']=(diff-dea)/dea
df['C/S']=(close/SMA20-1)*100
df['M/L']=(SMA60/SMA120-1)*100
df['S/M']=(SMA20/SMA60-1)*100
df.reset_index(drop=True, inplace=True)

df.loc[df['ts_code'] == df['ts_code'].shift(4),'Chg5D']= (df.close_qfq/df.close_qfq.shift(4)-1)*100
df.loc[df['ts_code'] == df['ts_code'].shift(9),'Chg10D']= (df.close_qfq/df.close_qfq.shift(9)-1)*100
df.loc[df['ts_code'] == df['ts_code'].shift(19),'Chg20D']= (df.close_qfq/df.close_qfq.shift(19)-1)*100
df.loc[df['ts_code'] == df['ts_code'].shift(59),'Chg60D']= (df.close_qfq/df.close_qfq.shift(59)-1)*100
df.loc[df['ts_code'] == df['ts_code'].shift(119),'Chg120D']= (df.close_qfq/df.close_qfq.shift(119)-1)*100

df['ATR'] = talib.ATR(df.high.values,df.low.values,df.pre_close.values, timeperiod=20)


df.fillna(0,inplace=True) #去除null
df.loc[(df['close_qfq']>df['EMA20']) & (df['close_qfq']>df['SMA20']) & (df['SMA20']>df['SMA20'].shift(19)),'kl1']=1 #破线拐头
df.loc[(df['EMA20']>df['EMA60']) & (df['close_qfq']>df['EMA20'].shift(19)),'K2']=1 #双均线交叉
df.loc[(df['EMA20']>df['SMA60']) & (df['close_qfq']>df['EMA20'].shift(19)),'K3']=1 
df.loc[(df['SMA20']>df['EMA60']) & (df['close_qfq']>df['SMA20'].shift(19)),'K4']=1 
df.loc[(df['SMA20']>df['SMA60']) & (df['close_qfq']>df['SMA20'].shift(19)),'K5']=1 

df.loc[(df['close_qfq']<df['EMA20']) & (df['close_qfq']<df['SMA20']) & (df['SMA20']<df['SMA20'].shift(19)),'kl1']=-1 #空头方向破线拐头
df.loc[(df['EMA20']<df['EMA60']) & (df['close_qfq']<df['EMA20'].shift(19)),'K2']=-1 #双均线交叉
df.loc[(df['EMA20']<df['SMA60']) & (df['close_qfq']<df['EMA20'].shift(19)),'K3']=-1 
df.loc[(df['SMA20']<df['EMA60']) & (df['close_qfq']<df['SMA20'].shift(19)),'K4']=-1 
df.loc[(df['SMA20']<df['SMA60']) & (df['close_qfq']<df['SMA20'].shift(19)),'K5']=-1 
df.fillna(0,inplace=True) #去除null
df['trendscore-L']=(df.kl1+df.K2+df.K3+df.K4+df.K5)

df.loc[(df['close_qfq']>df['EMA10']) & (df['close_qfq']>df['SMA10']) & (df['SMA10']>df['SMA10'].shift(19)),'ks1']=1 #破线拐头
df.loc[(df['EMA10']>df['EMA20']) & (df['close_qfq']>df['EMA10'].shift(19)),'KK2']=1 #双均线交叉
df.loc[(df['EMA10']>df['SMA20']) & (df['close_qfq']>df['EMA10'].shift(19)),'KK3']=1 
df.loc[(df['SMA10']>df['EMA20']) & (df['close_qfq']>df['SMA10'].shift(19)),'KK4']=1 
df.loc[(df['SMA10']>df['SMA20']) & (df['close_qfq']>df['SMA10'].shift(19)),'KK5']=1 

df.loc[(df['close_qfq']<df['EMA10']) & (df['close_qfq']<df['SMA10']) & (df['SMA10']<df['SMA10'].shift(19)),'ks1']=-1 #空头方向破线拐头
df.loc[(df['EMA10']<df['EMA20']) & (df['close_qfq']<df['EMA10'].shift(19)),'KK2']=-1 #双均线交叉
df.loc[(df['EMA10']<df['SMA20']) & (df['close_qfq']<df['EMA10'].shift(19)),'KK3']=-1 
df.loc[(df['SMA10']<df['EMA20']) & (df['close_qfq']<df['SMA10'].shift(19)),'KK4']=-1 
df.loc[(df['SMA10']<df['SMA20']) & (df['close_qfq']<df['SMA10'].shift(19)),'KK5']=-1 
df.fillna(0,inplace=True) #去除null
df['trendscore-S']=(df.ks1+df.KK2+df.KK3+df.KK4+df.KK5)
df = pd.merge(df, ISindex, how='left',on='ts_code') 

df=df[['date','ts_code','industry_name','open','close_qfq','high','low','amount','pct_chg','ATR','Chg5D','Chg20D','Chg60D','Chg120D','C/S','S/M','M/L','kl1','trendscore-L']]

df.head(10)


Wall time: 1min 34s


,date,ts_code,industry_name,open,close_qfq,high,low,amount,pct_chg,ATR,Chg5D,Chg20D,Chg60D,Chg120D,C/S,S/M,M/L,kl1,trendscore-L
0,2017-01-03,000001.SZ,银行,9.11,8.641158,9.18,9.09,420595.176,0.66,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-04,000001.SZ,银行,9.15,8.641158,9.18,9.14,411503.444,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-05,000001.SZ,银行,9.17,8.650591,9.18,9.15,315769.693,0.11,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-06,000001.SZ,银行,9.17,8.612857,9.17,9.11,327176.433,-0.44,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-09,000001.SZ,银行,9.13,8.631724,9.17,9.11,329994.604,0.22,0.0,-0.109170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2017-01-10,000001.SZ,银行,9.15,8.631724,9.16,9.14,220575.131,0.00,0.0,-0.109170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2017-01-11,000001.SZ,银行,9.14,8.622291,9.17,9.13,277553.207,-0.11,0.0,-0.327154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2017-01-12,000001.SZ,银行,9.13,8.631724,9.17,9.13,391869.402,0.11,0.0,0.219058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2017-01-13,000001.SZ,银行,9.14,8.641158,9.19,9.12,397601.906,0.11,0.0,0.109290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2017-01-16,000001.SZ,银行,9.15,8.622291,9.16,9.07,623025.820,-0.22,0.0,-0.109290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
%%time
df.ts_code=df.ts_code.str.split('.', 2).str[0]

df4=pd.merge(df,L3stocklist,how='left',on='ts_code')
#df4.drop(['index_code','level'],axis=1)

BBStock=df4[df4['ts_code'].notna()]
BBStock.fillna(0,inplace=True)
BBStock.loc[BBStock.close_qfq>=BBStock.open,'Signal']=0
BBStock.loc[BBStock.close_qfq<BBStock.open,'Signal']=1
BBStock['ChangeSignal']=BBStock.Signal.diff().abs()
BBStock.ChangeSignal.fillna(0,inplace=True) #解决首行0
BBStock['BBperiod']=BBStock.groupby('ts_code').ChangeSignal.cumsum()

#个股级别日线涨跌周期

high = BBStock.groupby(['ts_code','stock_name','BBperiod'])['high'].max().rename('high').reset_index()
low = BBStock.groupby(['ts_code','stock_name','BBperiod'])['low'].min().rename('low').reset_index()
amount = BBStock.groupby(['ts_code','stock_name','BBperiod'])['amount'].sum().rename('amount').reset_index()
open1 = BBStock.groupby(['ts_code','stock_name','BBperiod'])['open'].first().rename('open').reset_index()
close1 = BBStock.groupby(['ts_code','stock_name','BBperiod'])['close_qfq'].last().rename('close').reset_index()
lastperiod=BBStock.groupby(['ts_code','stock_name']).BBperiod.max().rename('lastbar')

BBS2 = pd.merge(open1,close1,how='left',on=('ts_code','stock_name','BBperiod')) 
BBS2 = pd.merge(BBS2,high,how='left',on=('ts_code','stock_name','BBperiod')) 
BBS2 = pd.merge(BBS2,low,how='left',on=('ts_code','stock_name','BBperiod')) 
BBS2 = pd.merge(BBS2,amount,how='left',on=('ts_code','stock_name','BBperiod')) 
BBS2 = pd.merge(BBS2,lastperiod,how='left',on=('ts_code','stock_name')) 

close=BBS2.close.values
#swsf.date=swsf.date.values

diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
BBS2['D-diff']=diff
BBS2['D-dea']=dea
BBS2['D-MACD']=my_macd
BBS2=BBS2[BBS2['BBperiod']==BBS2['lastbar']]
BBS2 = BBS2.sort_values(by=['D-MACD'],axis=0,ascending=[False])

#BBS2.to_csv('C:/Users/atianth/Desktop/BBtest2.csv',index=False,encoding='utf-8-sig') 
BBS2

Wall time: 38.9 s


,ts_code,stock_name,BBperiod,open,close,high,low,amount,lastbar,D-diff,D-dea,D-MACD
922523,688788,0,1.0,184.00,189.49,200.46,175.40,2412055.137,1.0,22.443976,5.908939,16.535038
921793,688536,0,9.0,253.71,294.59,303.50,245.81,1124493.726,9.0,71.010565,54.765332,16.245233
578850,300896,爱美客,12.0,493.00,481.99,506.00,480.20,546495.607,12.0,110.408898,94.618564,15.790334
908209,605111,0,2.0,131.04,124.98,131.49,122.82,887425.806,2.0,12.563532,1.768556,10.794977
577770,300842,0,37.0,61.00,67.88,70.60,60.89,637872.602,37.0,-26.718943,-36.372854,9.653910
...,...,...,...,...,...,...,...,...,...,...,...,...
914162,688095,0,20.0,280.98,280.01,287.95,280.01,76330.550,20.0,35.523411,49.373189,-13.849778
578863,300898,C熊猫,6.0,77.66,57.70,78.87,56.70,3950642.488,6.0,-44.941104,-27.196168,-17.744936
908068,605018,长华股份,8.0,16.25,15.01,16.58,14.95,174717.506,8.0,-17.573688,1.499925,-19.073612
859543,603444,0,186.0,470.93,470.25,474.99,462.88,359455.609,186.0,-22.608087,-2.211735,-20.396352


In [43]:
%%time
BBWStock=df4[df4['ts_code'].notna()]
BBWStock.fillna(0,inplace=True)
BBWStock['year']=BBWStock.date.dt.isocalendar().year
BBWStock['week']=BBWStock.date.dt.isocalendar().week
high = BBWStock.groupby(['ts_code','stock_name','year','week'])['high'].max().rename('high').reset_index()
low = BBWStock.groupby(['ts_code','stock_name','year','week'])['low'].min().rename('low').reset_index()
amount = BBWStock.groupby(['ts_code','stock_name','year','week'])['amount'].sum().rename('amount').reset_index()
open1 = BBWStock.groupby(['ts_code','stock_name','year','week'])['open'].first().rename('open').reset_index()
close1 = BBWStock.groupby(['ts_code','stock_name','year','week'])['close_qfq'].last().rename('close').reset_index()

BBWS2 = pd.merge(open1,close1,how='left',on=('ts_code','stock_name','year','week')) 
BBWS2 = pd.merge(BBWS2,high,how='left',on=('ts_code','stock_name','year','week')) 
BBWS2 = pd.merge(BBWS2,low,how='left',on=('ts_code','stock_name','year','week')) 
BBWS2 = pd.merge(BBWS2,amount,how='left',on=('ts_code','stock_name','year','week')) 

BBWS2.loc[BBWS2.close>=BBWS2.open,'Signal']=0
BBWS2.loc[BBWS2.close<BBWS2.open,'Signal']=1
BBWS2['ChangeSignal']=BBWS2.Signal.diff().abs()
BBWS2.ChangeSignal.fillna(0,inplace=True) #解决首行0
BBWS2['BBperiod']=BBWS2.groupby(['ts_code','stock_name']).ChangeSignal.cumsum()

high2 = BBWS2.groupby(['ts_code','stock_name','BBperiod'])['high'].max().rename('high').reset_index()
low2 = BBWS2.groupby(['ts_code','stock_name','BBperiod'])['low'].min().rename('low').reset_index()
amount2 = BBWS2.groupby(['ts_code','stock_name','BBperiod'])['amount'].sum().rename('amount').reset_index()
open2 = BBWS2.groupby(['ts_code','stock_name','BBperiod'])['open'].first().rename('open').reset_index()
close2 = BBWS2.groupby(['ts_code','stock_name','BBperiod'])['close'].last().rename('close').reset_index()
lastperiod=BBWS2.groupby(['ts_code','stock_name']).BBperiod.max().rename('lastbar')

BBWS3 = pd.merge(open2,close2,how='left',on=('ts_code','stock_name','BBperiod')) 
BBWS3 = pd.merge(BBWS3,high2,how='left',on=('ts_code','stock_name','BBperiod')) 
BBWS3 = pd.merge(BBWS3,low2,how='left',on=('ts_code','stock_name','BBperiod')) 
BBWS3 = pd.merge(BBWS3,amount2,how='left',on=('ts_code','stock_name','BBperiod')) 
BBWS3 = pd.merge(BBWS3,lastperiod,how='left',on=('ts_code','stock_name')) 

close=BBWS3.close.values
#swsf.date=swsf.date.values

diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
BBWS3['W-diff']=diff
BBWS3['W-dea']=dea
BBWS3['W-MACD']=my_macd
BBWS3=BBWS3[BBWS3['BBperiod']==BBWS3['lastbar']]
BBWS3 = BBWS3.sort_values(by=['W-MACD'],axis=0,ascending=[False])

BBWS3

Wall time: 23.2 s


,ts_code,stock_name,BBperiod,open,close,high,low,amount,lastbar,W-diff,W-dea,W-MACD
144942,300896,爱美客,3.0,410.11,481.99,506.00,410.00,3699517.805,3.0,72.655017,23.759187,48.895829
235398,688185,0,8.0,392.00,397.10,408.80,365.45,4030025.290,8.0,85.807508,55.928476,29.879032
236574,688536,0,5.0,253.71,294.59,303.50,245.81,1124493.726,5.0,51.876166,22.110492,29.765673
234736,688095,0,4.0,270.00,280.01,295.00,268.56,459369.874,4.0,58.222270,31.702259,26.520011
235314,688169,0,23.0,603.12,811.98,811.98,590.00,2258247.574,23.0,122.749518,99.259541,23.489976
...,...,...,...,...,...,...,...,...,...,...,...,...
236583,688551,0,3.0,38.10,39.30,42.21,38.10,445910.537,3.0,-6.779770,10.134877,-16.914647
233847,688013,0,5.0,36.13,35.67,39.30,35.00,350641.896,5.0,-25.169631,-3.778422,-21.391209
233355,605178,时空科技,5.0,62.71,60.11,62.71,60.11,253684.378,5.0,-5.510234,16.063743,-21.573977
236451,688418,0,5.0,36.97,32.13,38.50,31.53,401515.391,5.0,-26.213556,-0.759949,-25.453607


In [44]:
%%time
BBMStock=df4[df4['ts_code'].notna()]
BBMStock.fillna(0,inplace=True)
BBMStock['year']=BBMStock.date.dt.isocalendar().year
BBMStock['month']=BBMStock.date.dt.month
high = BBMStock.groupby(['ts_code','stock_name','year','month'])['high'].max().rename('high').reset_index()
low = BBMStock.groupby(['ts_code','stock_name','year','month'])['low'].min().rename('low').reset_index()
amount = BBMStock.groupby(['ts_code','stock_name','year','month'])['amount'].sum().rename('amount').reset_index()
open1 = BBMStock.groupby(['ts_code','stock_name','year','month'])['open'].first().rename('open').reset_index()
close1 = BBMStock.groupby(['ts_code','stock_name','year','month'])['close_qfq'].last().rename('close').reset_index()

BBMS2 = pd.merge(open1,close1,how='left',on=('ts_code','stock_name','year','month')) 
BBMS2 = pd.merge(BBMS2,high,how='left',on=('ts_code','stock_name','year','month')) 
BBMS2 = pd.merge(BBMS2,low,how='left',on=('ts_code','stock_name','year','month')) 
BBMS2 = pd.merge(BBMS2,amount,how='left',on=('ts_code','stock_name','year','month')) 

BBMS2.loc[BBMS2.close>=BBMS2.open,'Signal']=0
BBMS2.loc[BBMS2.close<BBMS2.open,'Signal']=1
BBMS2['ChangeSignal']=BBMS2.Signal.diff().abs()
BBMS2.ChangeSignal.fillna(0,inplace=True) #解决首行0
BBMS2['BBperiod']=BBMS2.groupby(['ts_code','stock_name']).ChangeSignal.cumsum()

high2 = BBMS2.groupby(['ts_code','stock_name','BBperiod'])['high'].max().rename('high').reset_index()
low2 = BBMS2.groupby(['ts_code','stock_name','BBperiod'])['low'].min().rename('low').reset_index()
amount2 = BBMS2.groupby(['ts_code','stock_name','BBperiod'])['amount'].sum().rename('Mamount').reset_index()
open2 = BBMS2.groupby(['ts_code','stock_name','BBperiod'])['open'].first().rename('open').reset_index()
close2 = BBMS2.groupby(['ts_code','stock_name','BBperiod'])['close'].last().rename('close').reset_index()
lastperiod=BBMS2.groupby(['ts_code','stock_name']).BBperiod.max().rename('lastbar')

BBMS3 = pd.merge(open2,close2,how='left',on=('ts_code','stock_name','BBperiod')) 
BBMS3 = pd.merge(BBMS3,high2,how='left',on=('ts_code','stock_name','BBperiod')) 
BBMS3 = pd.merge(BBMS3,low2,how='left',on=('ts_code','stock_name','BBperiod')) 
BBMS3 = pd.merge(BBMS3,amount2,how='left',on=('ts_code','stock_name','BBperiod')) 
BBMS3 = pd.merge(BBMS3,lastperiod,how='left',on=('ts_code','stock_name')) 
BBMS3 = BBMS3.sort_values(by=['ts_code','BBperiod'],axis=0,ascending=[True,True])
BBMS3['LastMonthK']=BBMS3.close.shift(1)/BBMS3.high.shift(1)
BBMS3['CurMonthK']=BBMS3.close/BBMS3.high


close=BBMS3.close.values
#swsf.date=swsf.date.values

diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
BBMS3['M-diff']=diff
BBMS3['M-dea']=dea
BBMS3['M-MACD']=my_macd
BBMS3=BBMS3[BBMS3['BBperiod']==BBMS3['lastbar']]
BBMS3 = BBMS3.sort_values(by=['M-MACD'],axis=0,ascending=[False])

BBMS3

Wall time: 14.1 s


,ts_code,stock_name,BBperiod,open,close,high,low,Mamount,lastbar,LastMonthK,CurMonthK,M-diff,M-dea,M-MACD
62704,688169,0,4.0,384.40,811.980000,811.98,376.00,2.128290e+07,4.0,0.908691,1.000000,107.728579,44.037837,63.690742
37594,300841,康华生物,1.0,850.03,462.000000,996.00,445.00,4.756648e+07,1.0,0.986767,0.463855,75.687843,19.919712,55.768131
4051,000661,长春高新,2.0,507.00,223.169245,513.50,318.51,1.425021e+08,2.0,0.968716,0.434604,67.485204,27.830820,39.654384
37313,300782,0,5.0,414.00,227.655481,425.00,402.10,1.105789e+06,5.0,0.968192,0.535660,81.917262,49.560599,32.356663
37707,300896,爱美客,0.0,320.00,481.990000,506.00,311.00,2.251311e+07,0.0,0.811203,0.952549,32.457556,4.925507,27.532049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37320,300783,三只松鼠,7.0,53.50,64.177468,65.77,53.50,3.517254e+06,7.0,0.591249,0.975786,3.798552,26.469296,-22.670744
62622,688116,0,6.0,41.50,32.285429,53.90,30.95,2.819940e+06,6.0,0.667273,0.598988,-2.238880,22.258699,-24.497580
62713,688179,0,1.0,48.00,43.300000,50.50,39.82,1.991774e+06,1.0,0.958150,0.857426,-15.732099,10.357215,-26.089315
62712,688178,0,5.0,37.00,37.090000,38.71,37.00,1.563417e+05,5.0,0.874733,0.958150,-11.534071,16.879544,-28.413615


In [45]:
StockReport=df4[['date','ts_code','stock_name','industry_name','close_qfq','Chg5D','Chg20D','Chg60D','Chg120D','C/S','S/M','M/L','kl1','trendscore-L']]
lastday=StockReport.date.max()
StockReport=StockReport[StockReport.date==lastday]
SDReport=BBS2[['ts_code','D-diff','D-dea','D-MACD']]
SWReport=BBWS3[['ts_code','W-diff','W-dea','W-MACD']]
SMReport=BBMS3[['ts_code','Mamount','M-diff','M-dea','M-MACD','LastMonthK','CurMonthK']]
StockReport=pd.merge(StockReport,SDReport,how='left',on=('ts_code'))
StockReport=pd.merge(StockReport,SWReport,how='left',on=('ts_code'))
StockReport=pd.merge(StockReport,SMReport,how='left',on=('ts_code'))

StockReport.to_csv('C:/quant/output/Stockoutput.csv',index=False,encoding='utf-8-sig') 
StockReport

,date,ts_code,stock_name,industry_name,close_qfq,Chg5D,Chg20D,Chg60D,Chg120D,C/S,...,D-MACD,W-diff,W-dea,W-MACD,Mamount,M-diff,M-dea,M-MACD,LastMonthK,CurMonthK
0,2020-10-30,000001,NaN,银行,17.75,0.282486,16.853193,33.058471,31.122079,6.119033,...,0.224161,0.706898,0.309871,0.397026,2.757857e+06,NaN,NaN,NaN,0.945154,0.972439
1,2020-10-30,000002,NaN,房地产,27.55,-1.607143,-0.253440,6.441630,8.792587,-0.906410,...,-0.249537,1.831423,2.141205,-0.309782,3.460997e+07,3.788401,3.798687,-0.010286,0.947903,0.956310
2,2020-10-30,000004,国农科技,医药生物,26.60,4.395604,15.351258,-7.252441,-8.307480,6.765137,...,0.511488,0.321308,1.096665,-0.775358,1.941602e+06,0.637216,0.377293,0.259923,0.666378,0.810549
3,2020-10-30,000005,世纪星源,公用事业,2.53,-4.166667,-2.316602,-13.356164,-5.597015,-4.021244,...,-0.027219,-0.102218,-0.099198,-0.003020,3.754836e+04,-3.043017,-3.741370,0.698353,0.821429,0.996774
4,2020-10-30,000006,NaN,房地产,5.72,-2.886248,-7.443366,-30.917874,18.602452,-8.296593,...,-0.058191,0.510925,0.501701,0.009224,2.537685e+06,-0.380493,-0.830223,0.449730,0.968373,0.714462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4038,2020-10-30,688598,NaN,NaN,113.00,-0.615655,10.286941,20.581470,0.000000,0.141350,...,-0.111097,16.916468,11.980718,4.935751,1.680487e+06,9.215210,2.155273,7.059936,0.915027,0.927903
4039,2020-10-30,688599,NaN,NaN,15.93,0.188679,1.724138,-6.073113,0.000000,-3.457471,...,0.436268,-11.795760,-11.458228,-0.337533,4.901449e+06,-6.445802,-0.686983,-5.758819,0.965802,0.886971
4040,2020-10-30,688600,NaN,NaN,20.01,-9.986505,-10.947931,-27.891892,0.000000,-10.293195,...,-0.197821,-5.830398,-7.483357,1.652959,6.153857e+06,-8.099511,-2.169488,-5.930023,0.886971,0.492857
4041,2020-10-30,688788,NaN,NaN,189.49,2.983696,0.000000,0.000000,0.000000,131.590912,...,16.535038,7.882291,-4.410227,12.292518,7.542350e+06,4.216922,-0.892206,5.109129,0.492857,0.758567
